# Wrangle HaluEval

Wrangles [HaluEval](https://github.com/RUCAIBox/HaluEval) QA data.

In [ ]:
import json
import os
from urllib.request import urlopen

import pandas as pd

pd.set_option("display.max_colwidth", None)

data_url = "https://raw.githubusercontent.com/RUCAIBox/HaluEval/main/data/qa_data.json"
records = []
with urlopen(data_url) as url:
    for line in url.readlines():
        records.append(json.loads(line))
df = pd.DataFrame(records)
df

In [ ]:
df = df.melt(
    id_vars=["knowledge", "question"],
    value_vars=["right_answer", "hallucinated_answer"],
    var_name="answer_type",
    value_name="answer",
)
df["is_hallucination"] = df["answer_type"] == "hallucinated_answer"
df = df.drop("answer_type", axis=1)
df = df.sort_values(["knowledge", "question"]).reset_index(drop=True)
df = df.rename(
    columns={"knowledge": "reference", "question": "query", "answer": "response"},
)
df

In [ ]:
output_path = "halueval_qa_data.jsonl"

try:
    os.remove(output_path)
except OSError:
    pass

with open(output_path, "a") as f:
    for record in df.to_dict(orient="records"):
        f.write(json.dumps(record) + "\n")